# GAN (Generative Adversarial Network ) ile Fashion MNIST veri seti ile mode ürün tasarımı.
- Generator(Üretici)ve Discriminator(Karar Verici)  olmak üzere iki ana bölümden oluşur

-  MNIST Training Data --> Sample(Real) Data  --> Ortak Discriminator
-  Gürültü --> Generator --> Sample(fake)Data -->

- Generator 'ün amacı Discriminator'ü kandıracak gerçeklektikte görüntü oluşturma

- MNIST Fashion -> 10 sınıf 28x28 boyutlu gri tonlamalı görüntülerden oluşur

# Proje Gerçekleştirme Aşamaları

- Kütüphanelerin Eklenmesi

- Veri Seti Yükleme

- Generator Modeli Tanımla: Fake Görüntüler Üretece
- Discriminator Modeli Tanımla: Gerçek ve Fake Görüntüler Arasında Ayrım Yapacak
- Loss Function Tanımla,optimizasyon algortiması tanımla
- Yardımcı Fonksiyonları Tanımla
- Eğitim Fonksiyonu Tanımla: Generator ve discriminator modelleri eğitilecek.



# 1-) Kütüphanlerin Eklenmesi

In [1]:
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import os
from tensorflow.keras.datasets import fashion_mnist



# 2-) Veri Setinin Yüklenmesi

In [2]:
BUFFER_SIZE =60000 # veri seti boyutu
BATCH_SIZE = 128 # batch boyutu

(train_images, _), (_, _) = fashion_mnist.load_data() # Görüntüleri al etiketleri kullanma

train_images = train_images.reshape(-1,28,28,1).astype("float32") # şekillendir ve floata çevir

train_images = (train_images-127.5)/ 127.5 # Normalize etme
train_dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(BUFFER_SIZE).batch(BATCH_SIZE) # veri setini shuffle et ve batch'le




# 3-) Generator Modelinin Hazırlanması

In [3]:
NOISE_DIM = 100

def make_generator_model():
    model = tf.keras.Sequential([
        layers.Dense(7*7*256, use_bias=False, input_shape=(NOISE_DIM,)),
        layers.BatchNormalization(),
        layers.LeakyReLU(),

        layers.Reshape((7, 7, 256)),

        # 7×7 → 14×14
        layers.Conv2DTranspose(128, (5,5), strides=(2,2), padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.LeakyReLU(),

        # 14×14 → 28×28
        layers.Conv2DTranspose(64, (5,5), strides=(2,2), padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.LeakyReLU(),

        # Final output (28×28×1)
        layers.Conv2DTranspose(1, (5,5), strides=(1,1), padding='same',
                               use_bias=False, activation='tanh')
    ])

    return model


# 4-) Discriminator Modelin Tasarlanması

In [4]:
IMG_SHAPE =(28,28,1)
def make_disc_model():
  model = tf.keras.Sequential([
      layers.Conv2D(64,(5,5) , strides=(2,2), padding ="same",input_shape=IMG_SHAPE),
      layers.LeakyReLU(),
      layers.Dropout(0.3),

      layers.Conv2D(128,(5,5),strides= (2,2),padding="same"),
      layers.LeakyReLU(),
      layers.Dropout(0.3),

      layers.Flatten(), # 3D 'yi düzleştir
      layers.Dense(1), # binary classification real/fake


  ])

  return model

disc = make_disc_model()
disc.build((None,28,28,1))
disc.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 14, 14, 64)     │         1,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu (LeakyReLU)         │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 7, 7, 128)      │       204,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_1 (LeakyReLU)       │ (None, 7, 7, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 7, 7, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 6272)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │         6,273 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 212,865 (831.50 KB)

 Trainable params: 212,865 (831.50 KB)

 Non-trainable params: 0 (0.00 B)

# 5-) Loss Function Tanımlama

In [11]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    return real_loss + fake_loss



generator = make_generator_model()
disc = make_disc_model()

gen_optimizer = tf.keras.optimizers.Adam(1e-4)
disc_optimizer = tf.keras.optimizers.Adam(1e-4)




/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


# 6-) Yardımcı Fonksiyonlar

In [12]:
seed = tf.random.normal([16,NOISE_DIM]) # sabit gürültü örneği

def generate_and_save_images(model,epoch,test_input):
  predictions = model(test_input,training=False)
  fig = plt.figure(figsize=(4,4))

  for i in range(predictions.shape[0]):
    plt.subplot(4,4,i+1)
    plt.imshow(predictions[i,:,:,0]*127.5+127.5,cmap="gray")
    plt.axis("off")

  if not os.path.exists("images"):
    os.makedirs("images")
  plt.savefig(f"generated_images/image_at_epoch{epoch:03d}.png")
  plt.close()





# 7-) Eğitim Fonksiyonları Tanımlama

In [14]:
def train(dataset, epochs):
    for epoch in range(epochs):
        gen_loss_total = 0
        disc_loss_total = 0
        batch_count = 0

        for image_batch in dataset:
            noise = tf.random.normal([image_batch.shape[0], NOISE_DIM])

            with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:

                # ---- generator ----
                generated_images = generator(noise, training=True)

                # ---- discriminator ----
                real_output = disc(image_batch, training=True)
                fake_output = disc(generated_images, training=True)

                # ---- losses ----
                g_loss = generator_loss(fake_output)
                d_loss = discriminator_loss(real_output, fake_output)

            # ---- gradients ----
            gradients_gen = gen_tape.gradient(g_loss, generator.trainable_variables)
            gradients_disc = disc_tape.gradient(d_loss, disc.trainable_variables)

            # ---- apply gradients ----
            gen_optimizer.apply_gradients(zip(gradients_gen, generator.trainable_variables))
            disc_optimizer.apply_gradients(zip(gradients_disc, disc.trainable_variables))

            gen_loss_total += g_loss
            disc_loss_total += d_loss
            batch_count += 1

        print(f"Epoch {epoch+1}/{epochs} | Gen Loss: {gen_loss_total/batch_count:.4f} | Disc Loss: {disc_loss_total/batch_count:.4f}")

        generate_and_save_images(generator, epoch+1, seed)
import os
os.makedirs("generated_images", exist_ok=True)
train(train_dataset,2)


Epoch 1/2 | Gen Loss: 1.0620 | Disc Loss: 1.1216
Epoch 2/2 | Gen Loss: 0.7106 | Disc Loss: 1.3866
